# Neural Network Demo

By Grant R. Vousden-Dishington

This notebook is heavily based on two blogs **A Neural Network in 11 lines of Python** ([Part 1](http://iamtrask.github.io/2015/07/12/basic-python-network/) & [Part 2](http://iamtrask.github.io/2015/07/27/python-network-part2/)) and [**Hinton's Droptout in 3 Lines of Python**](http://iamtrask.github.io/2015/07/28/dropout/), both by [*@iamtrask*](iamtrask.github.io)

In [1]:
import numpy as np

# Sigmoid function: "squashes" numbers into probabilities
def nonlin(x, deriv=False):
    x = x * (1-x) if deriv else 1 / (1 + np.exp(-x))
    return x

# Input: 4 training examples, 3-dimensional (4 x 3)
nnInput = np.array([
        [0, 0, 1],
        [0, 1, 1],
        [1, 0, 1],
        [1, 1, 1]
    ])

## 2-layer feed-forward network

In [2]:
# Output: expected classifications (4 x 1)
nnOutput2 = np.array([[0, 0, 1 ,1]]).T

# Initialize weights with mean of 0, seed random generator
np.random.seed(261015)
syn0 = 2 * np.random.random((3, 1)) - 1  # (3 x 1)

# Do full-batch training
for i in range(60000):
    
    # Forward propagation
    l0 = nnInput
    l1 = nonlin(l0 @ syn0)
    
    # Error: expected minus predicted (4 x 1)
    errl1 = nnOutput2 - l1
    
    # Calculate delta: the error times slope of the sigmoid at l1 values
    deltal1 = errl1 * nonlin(l1, deriv=True)
    
    # Update weights: matrix multiply the input times the delta values all at once
    syn0 += l0.T @ deltal1
    
print("Output after training")
print(l1)

Output after training
[[ 0.00390254]
 [ 0.00318154]
 [ 0.99740413]
 [ 0.99681544]]


## 3-layer back-propagating network

In [3]:
# Output: expected classifications (4 x 1)
nnOutput3 = np.array([[0, 1, 1 ,0]]).T

# Initialize weights with mean of 0, seed random generator
np.random.seed(261015)
syn0 = 2 * np.random.random((3, 4)) - 1  # (3 x 4)
syn1 = 2 * np.random.random((4, 1)) - 1  # (4 x 1)

# Do full-batch learning
for i in range(60000):
    
    # Forward Propagation
    l0 = nnInput
    l1 = nonlin(l0 @ syn0)
    l2 = nonlin(l1 @ syn1)
    
    # Output error: expected minus predicted (4 x 1)
    errl2 = nnOutput3 - l2
    
    # Getting a running view of the error every 10000 steps
    if not i % 10000:
        print("Error: " + str(np.mean(np.abs(errl2))))
        
    # Calculate delta for output layer: the error times slope of the sigmoid at l2 values (4 x 1)
    deltal2 = errl2 * nonlin(l2, deriv=True)
    
    # Hidden error: how much hidden layer contributed to output error (4 x 4)
    # THIS IS THE BACK-PROPAGATION STEP
    errl1 = deltal2 @ syn1.T
    
    # Calculate delta for hidden layer: the error times the sigmoid at l2 values (4 x 4)
    deltal1 = errl1 * nonlin(l1, deriv=True)
    
    # Update weights
    syn0 += l0.T @ deltal1
    syn1 += l1.T @ deltal2

Error: 0.500646983197
Error: 0.0108313939135
Error: 0.00721546010071
Error: 0.00571646037978
Error: 0.00485302997199
Error: 0.00427728962702


## 3-layer back-propagating network Mk. 2: Gradient Descent (alpha tuning)

In [4]:
# Output: expected classifications (4 x 1)
nnOutput3 = np.array([[0, 1, 1 ,0]]).T

alphas = [.001, .01, .1, 1, 10, 100, 1000]

# Try training with each different alpha-value
for a in alphas:
    # Initialize weights with mean of 0, seed random generator
    np.random.seed(271015)
    syn0 = 2 * np.random.random((3, 4)) - 1  # (3 x 4)
    syn1 = 2 * np.random.random((4, 1)) - 1  # (4 x 1)

    # This time, we track the previous weights for syn0 and syn1, as well as the direction of change
    syn0prev = np.zeros_like(syn0)
    syn1prev = np.zeros_like(syn1)
    syn0dir = np.zeros_like(syn0)
    syn1dir = np.zeros_like(syn1)
    
    print("Training with \u03B1 " + str(a))  # 03B1 is unicode for lowercase alpha
    
    # Do full-batch learning
    for i in range(60000):

        # Forward Propagation
        l0 = nnInput
        l1 = nonlin(l0 @ syn0)
        l2 = nonlin(l1 @ syn1)

        # Output error: expected minus predicted (4 x 1)
        errl2 = nnOutput3 - l2

        # Getting a running view of the error every 10000 steps
        if not i % 10000:
            print("Error: " + str(np.mean(np.abs(errl2))))

        # Calculate delta for output layer: the error times slope of the sigmoid at l2 values (4 x 1)
        deltal2 = errl2 * nonlin(l2, deriv=True)

        # Hidden error: how much hidden layer contributed to output error (4 x 4)
        # THIS IS THE BACK-PROPAGATION STEP
        errl1 = deltal2 @ syn1.T

        # Calculate delta for hidden layer: the error times the sigmoid at l2 values (4 x 4)
        deltal1 = errl1 * nonlin(l1, deriv=True)

        # Calculate the updates to the weights
        syn0update = l0.T @ deltal1
        syn1update = l1.T @ deltal2
        
        # For tracking, we see if the direction of the update has changed since the last step 
        syn0dir += (syn0update * syn0prev) < 0
        syn1dir += (syn1update * syn1prev) < 0
        
        # Update weights: this is where the alpha values come in
        syn0 += a * syn0update
        syn1 += a * syn1update
        
        syn0prev = syn0update
        syn1prev = syn1update
        
    # Print out the results for this alpha
    print("Syn0")
    print(syn0)
    print("Syn0 direction changes")
    print(syn0dir)

    print("Syn1")
    print(syn1)
    print("Syn1 direction changes")
    print(syn1dir)

Training with α 0.001
Error: 0.498898932827
Error: 0.498796948145
Error: 0.498686565246
Error: 0.498571609282
Error: 0.498448841313
Error: 0.498314805117
Syn0
[[ 0.58024654 -0.41866451  0.60108431 -0.67801471]
 [-0.70958078  0.19528945  0.80434397  0.37701121]
 [ 0.72880237 -0.61656649 -0.48386339 -0.36528227]]
Syn0 direction changes
[[ 0.  0.  1.  0.]
 [ 1.  0.  1.  1.]
 [ 0.  0.  1.  0.]]
Syn1
[[-0.57321094]
 [ 0.03002241]
 [ 0.03812072]
 [ 0.87185874]]
Syn1 direction changes
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
Training with α 0.01
Error: 0.498898932827
Error: 0.497337070621
Error: 0.489379108547
Error: 0.432236465616
Error: 0.303557999185
Error: 0.160599355169
Syn0
[[ 4.66593475 -1.75009237  4.69524699 -3.57501148]
 [-3.40666199  4.64328691  3.77424503  2.36547241]
 [ 1.79534168 -0.24637092 -0.80578793 -1.40299381]]
Syn0 direction changes
[[ 0.  3.  1.  0.]
 [ 1.  2.  1.  1.]
 [ 1.  3.  2.  1.]]
Syn1
[[-3.40358457]
 [-5.13222824]
 [ 5.99505957]
 [ 3.02751128]]
Syn1 direction changes
[[ 1.]

## 3-layer back-propagating network Mk. 3: Parameterizing the hidden layer size

For this dataset, the previous excercise demonstrated that an alpha of 10 worked best, and the hidden layer size was fixed at 4 x 1. In the original iamtrask post, this step only tested the performance of a network with 32 nodes. Instead of testing just one size, we'll try several here and keep the alpha constant.

In [5]:
# Output: expected classifications (4 x 1)
nnOutput3 = np.array([[0, 1, 1 ,0]]).T

# Set alpha and the hidden layer sizes we want to test
a = 10
l1sizes = [2, 4, 8, 16, 32, 64]

# Try training with each different alpha-value
for s in l1sizes:
    # Initialize weights with mean of 0, seed random generator
    # This is where the different hidden layer sizes come in
    np.random.seed(271015)
    syn0 = 2 * np.random.random((3, s)) - 1  # (3 x s)
    syn1 = 2 * np.random.random((s, 1)) - 1  # (s x 1)
    
    print("Training with {} hidden layer nodes ".format(s))
    
    # Do full-batch learning
    for i in range(60000):

        # Forward Propagation
        l0 = nnInput
        l1 = nonlin(l0 @ syn0)
        l2 = nonlin(l1 @ syn1)

        # Output error: expected minus predicted (4 x 1)
        errl2 = nnOutput3 - l2

        # Getting a running view of the error every 10000 steps
        if not i % 10000:
            print("Error: " + str(np.mean(np.abs(errl2))))

        # Calculate delta for output layer: the error times slope of the sigmoid at l2 values (4 x 1)
        deltal2 = errl2 * nonlin(l2, deriv=True)

        # Hidden error: how much hidden layer contributed to output error (4 x 4)
        # THIS IS THE BACK-PROPAGATION STEP
        errl1 = deltal2 @ syn1.T

        # Calculate delta for hidden layer: the error times the sigmoid at l2 values (4 x 4)
        deltal1 = errl1 * nonlin(l1, deriv=True)

        # Calculate the updates to the weights
        syn0 += a * (l0.T @ deltal1)
        syn1 += a * (l1.T @ deltal2)

Training with 2 hidden layer nodes 
Error: 0.500542514772
Error: 0.375678086516
Error: 0.375611385993
Error: 0.375562977412
Error: 0.375527817871
Error: 0.375502700424
Training with 4 hidden layer nodes 
Error: 0.498898932827
Error: 0.00277424125556
Error: 0.00194289672755
Error: 0.00157880649911
Error: 0.00136305538713
Error: 0.00121639933639
Training with 8 hidden layer nodes 
Error: 0.498573920102
Error: 0.00247201806838
Error: 0.00170567040317
Error: 0.00137593796584
Error: 0.00118242179508
Error: 0.00105172558577
Training with 16 hidden layer nodes 
Error: 0.496319067934
Error: 0.00230574062297
Error: 0.00159283504815
Error: 0.00128457712047
Error: 0.0011032857294
Error: 0.000980712209009
Training with 32 hidden layer nodes 
Error: 0.493817543054
Error: 0.00332116672913
Error: 0.0018904703995
Error: 0.00144658328493
Error: 0.00121176944923
Error: 0.00106133852859
Training with 64 hidden layer nodes 
Error: 0.495697784403
Error: 0.499999999843
Error: 0.499999999842
Error: 0.4999999

## 3-layer back-propagating network with Hinton's Dropout

The above simulations suggest our network works best with an alpha of 10 and a hidden layer size of 16 nodes. The network with 32 nodes doesn't do significantly worse, so we'll use that in the simulations below, so that our hidden layer is significantly larger than either the input or output layer. We'll see how training does with dropout rates of 10-50%.

In [30]:
# Output: expected classifications (4 x 1)
nnOutput3 = np.array([[0, 1, 1 ,0]]).T

# Set alpha and the hidden layer dropout rates
a = 10
l1size = 32
dropouts = [0.1, 0.2, 0.3, 0.4, 0.5]

# Try training with each different alpha-value
for d in dropouts:
    # Initialize weights with mean of 0, seed random generator
    # This is where the different hidden layer sizes come in
    np.random.seed(13)  # I changed the seed here for reasons explained in the next cell
    syn0 = 2 * np.random.random((3, l1size)) - 1  # (3 x 32)
    syn1 = 2 * np.random.random((l1size, 1)) - 1  # (32 x 1)
    
    print("Training with {}% hidden layer dropout ".format(int(100*d)))
    
    # Do full-batch learning
    for i in range(60000):

        # Forward Propagation
        l0 = nnInput
        l1 = nonlin(l0 @ syn0)
        
        # DROPOUT: Note the division by (1-d) to compensate for the dropped node activity
        l1dropped = l1 * np.random.binomial(1, 1.0-d, size=l1.shape) / (1.0 - d)
        l2 = nonlin(l1dropped @ syn1)

        # Output error: expected minus predicted (4 x 1)
        errl2 = nnOutput3 - l2

        # Getting a running view of the error every 10000 
        if not i % 10000:
            print("Error: " + str(np.mean(np.abs(errl2))))

        # Calculate delta for output layer: the error times slope of the sigmoid at l2 values (4 x 1)
        deltal2 = errl2 * nonlin(l2, deriv=True)

        # Hidden error: how much hidden layer contributed to output error (4 x 4)
        # THIS IS THE BACK-PROPAGATION STEP
        errl1 = deltal2 @ syn1.T

        # Calculate delta for hidden layer: the error times the sigmoid at l2 values (4 x 4)
        # In back-propagation, we do NOT drop nodes
        deltal1 = errl1 * nonlin(l1, deriv=True)

        # Calculate the updates to the weights
        syn0 += a * (l0.T @ deltal1)
        syn1 += a * (l1.T @ deltal2)

Training with 10% hidden layer dropout 
Error: 0.528716579972
Error: 2.20441872404e-19
Error: 1.54321083553e-14
Error: 1.91232415421e-18
Error: 7.02243940894e-25
Error: 2.57849416973e-30
Training with 20% hidden layer dropout 
Error: 0.539856694952
Error: 6.12735409176e-23
Error: 1.50692407773e-40
Error: 3.85085218196e-50
Error: 3.0852487622e-35
Error: 1.26822046033e-33
Training with 30% hidden layer dropout 
Error: 0.510231292419
Error: 0.500005334237
Error: 2.39094863462e-42
Error: 9.87170454361e-82
Error: 8.31397762445e-77
Error: 5.59973592673e-75
Training with 40% hidden layer dropout 
Error: 0.497345959473
Error: 5.1089134684e-70
Error: 3.54407439107e-114
Error: 0.25
Error: 1.59354836775e-124
Error: 8.95111618158e-184
Training with 50% hidden layer dropout 
Error: 0.550690223828
Error: 8.24443880427e-103
Error: 2.0994128787e-112
Error: 1.66267702286e-207
Error: 0.0
Error: 0.25


/usr/local/lib/python3.5/site-packages/IPython/kernel/__main__.py:5: RuntimeWarning: overflow encountered in exp


These are some interesting results. Dropout rates of 20-30% seem to push the network error astronomically low in just a few iterations. However, higher or lower dropout rates seem to reduce the performance of the network. This may be because the hidden layer is relatively small. When the number of hidden nodes is increased to 32, this effect disappears and all dropout rates result in networks performing much better than any of the networks without dropout.

Note: Some seeds seem to make networks stall in improvement (usually at 0.25 or 0.50 error). Changing the seed has a dramatic effect on this, so I think it's a problem with the setup of the problem or an implementation problem in the binomial distribution, not the dropout procedure.